#Web Scraping with Beautiful Soup

This tutorial will provide a very basic introduction to using the Beautiful Soup package to scrape text data from the web. 

##Installation

In a terminal, install Beautiful Soup if necessary by running <pre><code>conda install beautiful-soup</code></pre>

##Retrieving HTML

The general idea behind web scraping is to retrieve data that exists on a website, and convert it into a format that is usable for analysis. Webpages are rendered by the brower from HTML and CSS code, but much of the information included in the HTML underlying any website is not interesting to us.

We begin by reading in the source code for a given web page and creating a Beautiful Soup object with the BeautifulSoup function.

###urllib2
urllib2 is a module for working with urls, we will use it to open connections to urls and retrieve the webpage source.

In [1]:
from bs4 import BeautifulSoup
import urllib2
page = urllib2.urlopen('http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false').read()
soup = BeautifulSoup(page)
print type(soup)

<class 'bs4.BeautifulSoup'>


The soup object contains all of the HTML in the original document.

In [2]:
print soup.prettify()[0:1000]

<!DOCTYPE html>
<html xmlns:fb="http://www.facebook.com/2008/fbml">
 <head>
  <script src="http://sports-ak.espn.go.com/sports/optimizely.js">
  </script>
  <meta charset="utf-8"/>
  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <link href="http://a.espncdn.com/prod/assets/icons/E.svg" mask="" rel="icon" sizes="any"/>
  <meta content="#CC0000" name="theme-color"/>
  <script type="text/javascript">
   if(true && navigator && navigator.userAgent.toLowerCase().indexOf("teamstream") >= 0) {
        window.location = 'http://a.m.espn.go.com/mobilecache/general/apps/sc';
    }
  </script>
  <title>
   2014-15 Regular Season NBA Player Stats and League Leaders - Scoring Per Game - National Basketball Association - ESPN
  </title>
  <meta content="xuj1ODRluWa0frM-BjIr_aSHoUC7HB5C1MgmYAM_GkA" name="google-site-verification"/>
  <meta content="B1FEB7C682C46C8FCDA3130F3D18AC28" name="msvalidate.01"/>
  <meta content="noodp" name="googlebot"/>
  <meta content="index, follow" na

##Parsing HTML

By "parsing HTML", we mean pulling out only the relevant tags/attributes for our analysis.  What Beautiful Soup does is provide a handy bunch of methods for doing this efficiently.

###find method

The find method will search for and return the first tag matching your corresponding search criteria, if it exists.  You can specify tag and attribute info etc.  There is also a findAll method that will return a collection of tags matching your query.

In [3]:
table_div = soup.find(id='my-players-table')
print table_div.prettify()[0:4000]

<div class="col-main" id="my-players-table">
 <div class="mod-container mod-table">
  <div class="mod-header stathead">
   <h4>
    Points Per Game Leaders - All Players
   </h4>
  </div>
  <div class="mod-content">
   <table cellpadding="3" cellspacing="1" class="tablehead">
    <tr align="right" class="colhead">
     <td align="left" style="width:20px;">
      RK
     </td>
     <td align="left">
      PLAYER
     </td>
     <td align="left">
      TEAM
     </td>
     <td>
      <a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/gamesPlayed/qualified/false" title="Games Played">
       GP
      </a>
     </td>
     <td>
      <a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgMinutes/qualified/false" title="Minutes Per Game">
       MPG
      </a>
     </td>
     <td>
      <a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/order/false" title="Points Per Game">
       PTS

Notice that the object returned by the find method is just another inner HTML structure, which we can step through just like we would have with the original soup object.  We've gone to the location in the webpage where the table that we seek starts, now we can use find again to get to the table data.

In [4]:
table = table_div.find("table")
print table

<table cellpadding="3" cellspacing="1" class="tablehead">
<tr align="right" class="colhead"><td align="left" style="width:20px;">RK</td><td align="left">PLAYER</td><td align="left">TEAM</td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/gamesPlayed/qualified/false" title="Games Played">GP</a></td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgMinutes/qualified/false" title="Minutes Per Game">MPG</a></td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/order/false" title="Points Per Game">PTS</a></td><td><span title="Field Goals Made-Attempted Per Game">FGM-FGA</span></td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/fieldGoalPct/qualified/false" title="Field Goal Percentage">FG%</a></td><td><span title="3-Point Field Goals Made-Attempted Per Game">3PM-3PA</span></td><td><a href="http://espn.go.com/nba/statistics/play

###Searching by Attributes

Now that we have the table object, we need to step through the rows.  First we'll find the header row so we can populate what the field names will be in our data.  Here we're searching for tags under the table tag whose class attritbute is "colhead".  

In [5]:
table_head = table.find(attrs={"class":'colhead'})
print table_head.prettify()

<tr align="right" class="colhead">
 <td align="left" style="width:20px;">
  RK
 </td>
 <td align="left">
  PLAYER
 </td>
 <td align="left">
  TEAM
 </td>
 <td>
  <a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/gamesPlayed/qualified/false" title="Games Played">
   GP
  </a>
 </td>
 <td>
  <a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgMinutes/qualified/false" title="Minutes Per Game">
   MPG
  </a>
 </td>
 <td>
  <a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/order/false" title="Points Per Game">
   PTS
  </a>
 </td>
 <td>
  <span title="Field Goals Made-Attempted Per Game">
   FGM-FGA
  </span>
 </td>
 <td>
  <a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/fieldGoalPct/qualified/false" title="Field Goal Percentage">
   FG%
  </a>
 </td>
 <td>
  <span title="3-Point Field Goals Made-Attempted Per Game">
   3PM-3PA
  </span>
 </td>
 <t

Now we find the actual values by searching for the 'td' tags, which is the tag for table data.

In [6]:
header_cols = table_head.findAll('td')
print header_cols

[<td align="left" style="width:20px;">RK</td>, <td align="left">PLAYER</td>, <td align="left">TEAM</td>, <td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/gamesPlayed/qualified/false" title="Games Played">GP</a></td>, <td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgMinutes/qualified/false" title="Minutes Per Game">MPG</a></td>, <td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/order/false" title="Points Per Game">PTS</a></td>, <td><span title="Field Goals Made-Attempted Per Game">FGM-FGA</span></td>, <td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/fieldGoalPct/qualified/false" title="Field Goal Percentage">FG%</a></td>, <td><span title="3-Point Field Goals Made-Attempted Per Game">3PM-3PA</span></td>, <td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/threePointFieldGoalPct/qualified/false" t

Finally, we step through these columns and save them to a list to be used later.  We'll ignore the rank column (RK) because that doesn't give us anything we want later.  We also separate the **PLAYER** column into **PLAYER** and **POSITION**.

In [7]:
cols = []
for header_col in header_cols:
    val = header_col.string
    if val != 'RK':
        cols.append(val)
    if val == 'PLAYER':
        cols.append('POSITION')
print cols

[u'PLAYER', 'POSITION', u'TEAM', u'GP', u'MPG', u'PTS', u'FGM-FGA', u'FG%', u'3PM-3PA', u'3P%', u'FTM-FTA', u'FT%']


###Stepping Through a Table

The table rows are indicated by the tag 'tr'.  Again we can find them all and iterate through them.  Within each row we iterate through the respective columns.

In [8]:
table_rows = table.findAll('tr')
print table_rows

[<tr align="right" class="colhead"><td align="left" style="width:20px;">RK</td><td align="left">PLAYER</td><td align="left">TEAM</td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/gamesPlayed/qualified/false" title="Games Played">GP</a></td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgMinutes/qualified/false" title="Minutes Per Game">MPG</a></td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/order/false" title="Points Per Game">PTS</a></td><td><span title="Field Goals Made-Attempted Per Game">FGM-FGA</span></td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/fieldGoalPct/qualified/false" title="Field Goal Percentage">FG%</a></td><td><span title="3-Point Field Goals Made-Attempted Per Game">3PM-3PA</span></td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/threePointFieldGoalPct/qu

We will save our results in 2 different ways to demonstrate how we can handle both.  The first will be a list of dicts where the key is the field name and the value is the field value.  The second will just be a list of lists of stats with no field name values (we've already defined them earlier).

In [9]:
players_stats_dicts = []
players_stats_array = []
for row in table_rows:
    if row.attrs['class'][0]=='colhead':
        continue
    player_stats = []
    row_cols = row.find_all('td')
    col_vals = []
    player_col = row_cols[1]
    player_name = player_col.find('a').string
    player_position = player_col.contents[1]
    player_position = player_position.split(' ')[1]
    player_stats.append(player_name)
    player_stats.append(player_position)
    for i in range(2, len(row_cols)):
        stat = row_cols[i].string
        player_stats.append(stat)
    players_stats_array.append(player_stats)
    player_stats = dict(zip(cols, player_stats))
    players_stats_dicts.append(player_stats)
print players_stats_dicts[0:5]        

[{u'FGM-FGA': u'9.4-22.0', u'MPG': u'34.4', u'FTM-FTA': u'8.1-9.8', u'FG%': u'.426', u'GP': u'67', u'PLAYER': u'Russell Westbrook', u'FT%': u'.835', u'TEAM': u'OKC', u'3PM-3PA': u'1.3-4.3', 'POSITION': u'PG', u'PTS': u'28.1', u'3P%': u'.299'}, {u'FGM-FGA': u'8.0-18.1', u'MPG': u'36.8', u'FTM-FTA': u'8.8-10.2', u'FG%': u'.440', u'GP': u'81', u'PLAYER': u'James Harden', u'FT%': u'.868', u'TEAM': u'HOU', u'3PM-3PA': u'2.6-6.9', 'POSITION': u'SG', u'PTS': u'27.4', u'3P%': u'.375'}, {u'FGM-FGA': u'8.8-17.3', u'MPG': u'33.8', u'FTM-FTA': u'5.4-6.3', u'FG%': u'.510', u'GP': u'27', u'PLAYER': u'Kevin Durant', u'FT%': u'.854', u'TEAM': u'OKC', u'3PM-3PA': u'2.4-5.9', 'POSITION': u'SF', u'PTS': u'25.4', u'3P%': u'.403'}, {u'FGM-FGA': u'9.0-18.5', u'MPG': u'36.1', u'FTM-FTA': u'5.4-7.7', u'FG%': u'.488', u'GP': u'69', u'PLAYER': u'LeBron James', u'FT%': u'.710', u'TEAM': u'CLE', u'3PM-3PA': u'1.7-4.9', 'POSITION': u'SF', u'PTS': u'25.3', u'3P%': u'.354'}, {u'FGM-FGA': u'9.4-17.6', u'MPG': u'36.1'

Here we've used the zip function to combine pairs of lists into tuples, and then transformed that into a dict to get a dictionary of FIELD --> VALUE for every player in the table.

Beautiful Soup has many other features, including the ability to step up, down, and sideways in the HTML tree and basically search for any tags, attributes, or values.  For more, take a look at the [Documentation](http://www.crummy.com/software/BeautifulSoup/bs4/doc/)

##Load into Pandas

Let's load our scraped data into Pandas and take a look at it.  Here is the first way we can do it, simply directly from the dictionary we defined.

In [10]:
import pandas as pd
import numpy as np

df = pd.DataFrame.from_dict(players_stats_dicts)
df.head()

,3P%,3PM-3PA,FG%,FGM-FGA,FT%,FTM-FTA,GP,MPG,PLAYER,POSITION,PTS,TEAM
0,.299,1.3-4.3,.426,9.4-22.0,.835,8.1-9.8,67,34.4,Russell Westbrook,PG,28.1,OKC
1,.375,2.6-6.9,.440,8.0-18.1,.868,8.8-10.2,81,36.8,James Harden,SG,27.4,HOU
2,.403,2.4-5.9,.510,8.8-17.3,.854,5.4-6.3,27,33.8,Kevin Durant,SF,25.4,OKC
3,.354,1.7-4.9,.488,9.0-18.5,.710,5.4-7.7,69,36.1,LeBron James,SF,25.3,CLE
4,.083,0.0-0.2,.535,9.4-17.6,.805,5.5-6.8,68,36.1,Anthony Davis,PF,24.4,NO


Now here is a 2nd way we can do it.  We convert the 2D stats array into a numpy array and create a Pandas dataframe from it along with the list of column headers we defined earlier.

In [11]:
np_array = np.array(players_stats_array)
df = pd.DataFrame(np_array, columns=cols)
df.head()

,PLAYER,POSITION,TEAM,GP,MPG,PTS,FGM-FGA,FG%,3PM-3PA,3P%,FTM-FTA,FT%
0,Russell Westbrook,PG,OKC,67,34.4,28.1,9.4-22.0,.426,1.3-4.3,.299,8.1-9.8,.835
1,James Harden,SG,HOU,81,36.8,27.4,8.0-18.1,.440,2.6-6.9,.375,8.8-10.2,.868
2,Kevin Durant,SF,OKC,27,33.8,25.4,8.8-17.3,.510,2.4-5.9,.403,5.4-6.3,.854
3,LeBron James,SF,CLE,69,36.1,25.3,9.0-18.5,.488,1.7-4.9,.354,5.4-7.7,.710
4,Anthony Davis,PF,NO,68,36.1,24.4,9.4-17.6,.535,0.0-0.2,.083,5.5-6.8,.805


##Exercise

The goal of this exercise is to combine the scoring and assists statistics for every player in the NBA in 2014-2015.  The end result will have them in a pandas dataframe with the fields from both pages for every player.

The general steps should be as follows:
- Create a function get_cols that retrieves the names of the header columns given a table element (skip the ranks, split the positions)
- Create a function get_data that retrieves the actual table data given a table element (skip the ranks, split the positions).  You can use either the dict approach or the numpy array approach.
- Write a python loop to loop through the various pages and call these functions on the appropriate urls so that you can retrieve every player (rather than just the top few).
- Repeat the above on both the scoring and assists URLs to get a pandas dataframe for both of them
- Use the pandas.DataFrame.join() function to join your 2 pandas dataframes together and get a total result

### Function get_cols that grabs the column names [skips the rank column name, and splits the positions]

In [12]:
def get_cols(table):
    table_head = table.find(attrs={"class":'colhead'})
    header_cols = table_head.findAll('td')
    cols = []
    for header_col in header_cols:
        val = header_col.string
        if val != 'RK':
            cols.append(val)
        if val == 'PLAYER':
            cols.append('POSITION')
    return cols

### Function get_data returns array with the actual table data

In [13]:
def get_data(table):
    table_rows = table.findAll('tr')
    players_stats_array = []
    for row in table_rows:
        if row.attrs['class'][0]=='colhead':
            continue
        player_stats = []
        row_cols = row.find_all('td')
        col_vals = []
        player_col = row_cols[1]
        player_name = player_col.find('a').string
        player_position = player_col.contents[1]
        player_position = player_position.split(' ')[1]
        player_stats.append(player_name)
        player_stats.append(player_position)
        for i in range(2, len(row_cols)):
            stat = row_cols[i].string
            player_stats.append(stat)
        players_stats_array.append(player_stats)
    return players_stats_array

### Get the final page number

In [46]:
final_page = int(soup.find(class_='page-numbers').contents[0].split(' of ')[-1])

### Populate the data in an array

In [47]:
Final_data=[]
counter=1
for i in range(final_page):
    url= 'http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/count/{}'.format(counter)
    counter+=40
    page = urllib2.urlopen(url).read()
    soup = BeautifulSoup(page)
    Final_data.extend(get_data(soup.find(id='my-players-table').find("table")))
    

### Place the data in a pandas DataFrame

In [48]:
np_array_final = np.array(Final_data)
df_final = pd.DataFrame(np_array_final, columns=cols)
df_final.tail()

,PLAYER,POSITION,TEAM,GP,MPG,PTS,FGM-FGA,FG%,3PM-3PA,3P%,FTM-FTA,FT%
487,Malcolm Lee,SG,PHI,1,2.0,0.0,0.0-1.0,.000,0.0-0.0,.000,0.0-0.0,.000
488,Kalin Lucas,PG,MEM,1,6.0,0.0,0.0-1.0,.000,0.0-0.0,.000,0.0-0.0,.000
489,Seth Curry,SG,PHX,2,4.0,0.0,0.0-1.5,.000,0.0-0.5,.000,0.0-0.0,.000
490,David Wear,PF,SAC,2,3.5,0.0,0.0-1.0,.000,0.0-0.5,.000,0.0-0.0,.000
491,Jerrelle Benimon,PF,UTAH,2,1.5,0.0,0.0-0.0,.000,0.0-0.0,.000,0.0-0.0,.000


### Repeat Steps for scoring and assists data

In [49]:
scoring_data = []
counter=1
for i in range(final_page):
    url= 'http://espn.go.com/nba/statistics/player/_/stat/scoring/qualified/false/count/{}'.format(counter)
    counter+=40
    page = urllib2.urlopen(url).read()
    soup = BeautifulSoup(page)
    scoring_data.extend(get_data(soup.find(id='my-players-table').find("table")))

In [50]:
np_scoring_array = np.array(scoring_data)
df_scoring = pd.DataFrame(np_scoring_array, columns=cols)
df_scoring.tail()

,PLAYER,POSITION,TEAM,GP,MPG,PTS,FGM-FGA,FG%,3PM-3PA,3P%,FTM-FTA,FT%
487,Malcolm Lee,SG,PHI,1,2.0,0,0-1,.000,0-0,.000,0-0,.000
488,Kalin Lucas,PG,MEM,1,6.0,0,0-1,.000,0-0,.000,0-0,.000
489,Seth Curry,SG,PHX,2,4.0,0,0-3,.000,0-1,.000,0-0,.000
490,David Wear,PF,SAC,2,3.5,0,0-2,.000,0-1,.000,0-0,.000
491,Jerrelle Benimon,PF,UTAH,2,1.5,0,0-0,.000,0-0,.000,0-0,.000


In [52]:
assists_data = []
counter=1
for i in range(final_page):
    url= 'http://espn.go.com/nba/statistics/player/_/stat/assists/qualified/false/count/{}'.format(counter)
    counter+=40
    page = urllib2.urlopen(url).read()
    soup = BeautifulSoup(page)
    assists_data.extend(get_data(soup.find(id='my-players-table').find("table")))


In [61]:
col_assists=['PLAYER','POSITION','TEAM','GP','MPG','AST','APG','TO','TOPG','AP48M','AST/TO']
np_assists_array = np.array(assists_data)
df_assists = pd.DataFrame(np_assists_array, columns=col_assists)
df_assists.tail()

,PLAYER,POSITION,TEAM,GP,MPG,AST,APG,TO,TOPG,AP48M,AST/TO
487,Jerrelle Benimon,PF,UTAH,2,1.5,0,0.0,1,0.5,0.0,0.00
488,Eric Moreland,PF,SAC,3,0.7,0,0.0,0,0.0,0.0,0.00
489,Miroslav Raduljica,C,MIN,5,4.6,0,0.0,2,0.4,0.0,0.00
490,Julius Randle,PF,LAL,1,14.0,0,0.0,1,1.0,0.0,0.00
491,Bruno Caboclo,SF,TOR,8,2.9,0,0.0,4,0.5,0.0,0.00


### Joining the two dataframes

In [65]:
df_joined= df_scoring.merge(df_assists)

In [67]:
df_joined.head()

,PLAYER,POSITION,TEAM,GP,MPG,PTS,FGM-FGA,FG%,3PM-3PA,3P%,FTM-FTA,FT%,AST,APG,TO,TOPG,AP48M,AST/TO
0,James Harden,SG,HOU,81,36.8,2217,647-1470,.440,208-555,.375,715-824,.868,565,7.0,321,4.0,9.1,1.76
1,Stephen Curry,PG,GS,80,32.7,1900,653-1341,.487,286-646,.443,308-337,.914,619,7.7,249,3.1,11.4,2.49
2,Russell Westbrook,PG,OKC,67,34.4,1886,627-1471,.426,86-288,.299,546-654,.835,574,8.6,293,4.4,12.0,1.96
3,LeBron James,SF,CLE,69,36.1,1743,624-1279,.488,120-339,.354,375-528,.710,511,7.4,272,3.9,9.8,1.88
4,Damian Lillard,PG,POR,82,35.7,1720,590-1360,.434,196-572,.343,344-398,.864,507,6.2,222,2.7,8.3,2.28
